In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json
from spacy.util import filter_spans

In [2]:
# Define the path to the training data file
train_file="../data/annotations (5).json"

# Load the training data from the JSON file
with open(train_file) as f:
    data = json.load(f)

# Extract the relevant columns from the DataFrame
intent_df = pd.DataFrame(data)
intent_df['annotations']
intent_df = intent_df.join(intent_df['annotations'].apply(pd.Series))


intent_df.rename(columns={1: 'entities'}, inplace=True)
intent_df.rename(columns={0: 'pattern'}, inplace=True)

# Drop the unneccessary column
intent_df.drop(columns=['annotations'], inplace=True)


In [3]:
# Define a function to extract entities from the 'entities' column
def extract_entities(entities):
    if isinstance(entities, dict) :
        return [tuple(entity) for entity in entities['entities']]
    return np.nan

# Apply the function to extract entities
intent_df['entities'] = intent_df['entities'].apply(extract_entities)

# Drop rows with missing entities
intent_df=intent_df.dropna()


In [4]:
training_data=intent_df
training_data

,pattern,entities
0,Can you generate a bar chart showing sales per...,"[(19, 28, VISUALIZATION), (37, 42, SALES), (58..."
1,Can you generate a bar chart showing sales per...,"[(19, 28, VISUALIZATION), (37, 42, SALES), (58..."
2,Show me a bar chart of sales performance by re...,"[(10, 19, VISUALIZATION), (23, 28, SALES), (44..."
3,I'd like to see a bar chart of sales performan...,"[(18, 27, VISUALIZATION), (31, 36, SALES), (52..."
4,I'd like to see a bar chart of sales performan...,"[(18, 27, VISUALIZATION), (31, 36, SALES), (52..."
...,...,...
199,I need a bar chart of order priority Low by ca...,"[(9, 18, VISUALIZATION), (22, 36, ORDER_PRIORI..."
200,I need a bar chart of priority Medium by produ...,"[(9, 18, VISUALIZATION), (22, 30, ORDER_PRIORI..."
201,I need a bar chart of priority Critical by pro...,"[(9, 18, VISUALIZATION), (22, 30, ORDER_PRIORI..."
202,I need a bar chart of priority High by product...,"[(9, 18, VISUALIZATION), (22, 30, ORDER_PRIORI..."


In [5]:
# Create a blank spaCy model
nlp = spacy.blank("en")

# Create a DocBin object to store the training data
doc_bin = DocBin()

In [6]:
import os
# Ensure the directory exists
os.makedirs("../data/ner_model", exist_ok=True)



In [7]:
# Iterate over the training data and create spaCy documents
for index, training_example in tqdm(intent_df.iterrows(), total=intent_df.shape[0]):
    text = training_example['pattern']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.set_ents(filtered_ents)
    doc_bin.add(doc)
#  Save the training data to a file
doc_bin.to_disk("../data/ner_model/train.spacy")

100%|██████████| 171/171 [00:00<00:00, 1925.68it/s]


In [8]:
# Initialize a spaCy config file
! python -m spacy init config ../data/ner_model/config.cfg --lang en --pipeline ner --optimize efficiency

# Train the spaCy model
! python -m spacy train ../data/ner_model/config.cfg --output ../data/ner_model/ --paths.train ../data/ner_model/train.spacy --paths.dev ../data/ner_model/train.spacy



✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.

ℹ Saving to output directory: ../data/ner_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     67.58    0.00    0.00    0.00    0.00
 10     200        136.19   3408.69   98.86   98.70   99.02    0.99
 22     400         82.61    211.20   99.84   99.67  100.00    1.00
 37     600         66.17     90.76   99.84   99.67  100.00    1.00
 55     800         98.52     98.92   99.84   99.67  100.00    1.00
 78    1000        203.75    133.73   99.84   99.67  100.00    1.00
105    1200        468.04    170.52   99.84  

In [9]:
# Load the trained spaCy model
nlp_trained_model = spacy.load("../data/ner_model/model-best")

In [16]:
# Create a spaCy document from the input text
doc = nlp_trained_model('''
Show me a scatter plot of country  and product type Bike.
''')

In [17]:
# Visualize the entities in the document
spacy.displacy.render(doc, style="ent", jupyter=True)